In [1]:
# System 
#This script computes the HWMI based on daily fields for a specific period in cross-validation. 
import matplotlib
from scipy import signal
import os
import datetime
import time
import copy
import shutil
import sys
from function_read import *
import numpy as np
from scipy.stats import gaussian_kde
import xarray as xr
import calendar
import locale
from netCDF4 import num2date, date2num
from joblib import Parallel, delayed
import joblib
from netCDF4 import num2date, date2num, Dataset
from numpy import dtype

In [5]:
var='tmax'
expname='ERA5'
memb_str='0' # there are not members for ERA5
nrealisation=1
# Region #
reg_name ='Europe'
min_lon = -15
min_lat = 25
max_lon = 60
max_lat = 70
lats_bnds = np.array([min_lat,max_lat])
lons_bnds = np.array([min_lon, max_lon])
# Season #
season='15MJJA'
season_start_day=[5,15]
season_start_day=[8,31]

# nday=109 #15 of May to 31st of Aug
nday=365
# Period #
ref_year1=1981
ref_year2=1983
nyear=len(range(ref_year1,ref_year2))+1

In [6]:
low=False # if data with 1 degree resolution is used (True) or not (False)

if (low==True):
    # path="/work/csp/vt17420/ERA5/daily/russo/"+var+"_r360x180/"
    # Where the outputs will be saved
    # dirout="/work/csp/vt17420/ERA5/daily/russo/"+var+"_r360x180/HWMI/"
    path="/Users/nilshempelmann/birdhouse/testdata/"
    dirout="/Users/nilshempelmann/birdhouse/testdata/"
    lons_reg=np.arange(min_lon,max_lon,1)
    lats_reg=np.arange(min_lat,max_lat,1)
    nlon=len(lons_reg)
    nlat=len(lats_reg)
else:
    # path="/work/csp/vt17420/ERA5/daily/global/"+var+"/"
    # dirout="/data/csp/vt17420/CLINT_PRODUCTS/"+expname+"_HWMI_"+var+"/"
    path="/Users/nilshempelmann/birdhouse/testdata/"
    dirout="/Users/nilshempelmann/birdhouse/testdata/"
    lons_reg=np.arange(min_lon,max_lon+0.25,0.25)
    lats_reg=np.arange(min_lat,max_lat+0.25,0.25)
    nlon=len(lons_reg)
    nlat=len(lats_reg)

In [7]:
file = '/Users/nilshempelmann/birdhouse/testdata/download_2015.nc'

# data=np.zeros((nlon,nlat,nyear,nday,1)) # daily data for all the years
data=np.zeros((241,181,365,nday,1))
for iyear,year in enumerate(range(ref_year1,ref_year2)):
    days_may=np.linspace(15, 31, num=17)
    # obs1=xr.open_dataset(path+var+"_"+str(year)+'05.nc')
    obs1=xr.open_dataset(file)
    if low==False:
        obs1 = obs1.sel(latitude=slice(max_lat,min_lat),longitude=slice(min_lon,max_lon))
    obs=obs1.sel(time=obs1.time.time.dt.day.isin(days_may))
    # selection of the days in june, july and august
    for mon in range(6,9):
        # obs1=xr.open_dataset(path+var+"_"+str(year)+str(mon).zfill(2)+'.nc')
        obs1=xr.open_dataset(file)
        if low==False:
            obs1 = obs1.sel(latitude=slice(max_lat,min_lat),longitude=slice(min_lon,max_lon))
        obs=xr.merge(([obs,obs1]))
    data[:,:,iyear,:,0]=np.transpose(obs.to_array()[0,:,:,:],[2,1,0])

In [8]:
### PERCENTILE THRESHOLD
#percent_thresh = 95
percent_thresh = 90
cv='CV'
cv_str = cv
### MINIMAL DURATION OF A HW
#duration_min = 5
duration_min = 3

In [10]:
from HWMI import *

In [19]:
nyear = 365

In [20]:
nmemb=data.shape[4]
ndayseas = nday//duration_min +1
HWMI = np.zeros((nyear,nmemb,nlat,nlon))
ndayexedthreshold = np.zeros((nyear,nmemb,nlat,nlon))
subHWarray = np.zeros((nyear, ndayseas, nmemb, nlat, nlon))
#HW = np.zeros((nyear,nmemb,nday,nlat,nlon))
#DDthreshold = np.zeros((nyear,nmemb,nlat,nlon))
#fitsubHWarray = np.zeros((nyear, ndayseas, nmemb, nlat, nlon))
#impossible_fit_list = []
#sstMeanarray = np.zeros((nyear,nmemb,nlat,nlon))

def parallelized_HWMIs_computation(ilat, ilon):    #, HWMI, ndayexedthreshold, DDthreshold, fitsubHWarray, subHWarray):
        HWMIyear, HWlstyear, HWstartmembyear, HWendmembyear, ndayexedthresholdyear, DDthresholdyear, subHWarrayyear, fitsubHWarrayyear, sstMeanarrayyear, impossible_fit = calc_HWMIyear(data[ilon,ilat,:,:,:], cross_valid = cv, percent_thresh = percent_thresh, duration_min = duration_min)
        #print HWMIyear
        HWMI[:,:,ilat,ilon]=np.array(HWMIyear)
        ndayexedthreshold[:,:,ilat,ilon]=np.array(ndayexedthresholdyear)
        subHWarray[:,:, :, ilat,ilon]=np.array(subHWarrayyear)
        #DDthreshold[:,:,ilat,ilon]=np.array(DDthresholdyear)
        #fitsubHWarray[:,:, :, ilat,ilon]=np.array(fitsubHWarrayyear)
        #sstMeanarray[:,:,ilat,ilon]=np.array(sstMeanarrayyear)
        #impossible_fit_list.append((impossible_fit,ilat,ilon))
        return()

Parallel(n_jobs=-1, timeout = 5*3600, verbose = 20, require='sharedmem', mmap_mode='w+')(delayed(parallelized_HWMIs_computation)(ilat, ilon) for ilat in range(nlat) for ilon in range(nlon))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapse

IndexError: index 241 is out of bounds for axis 0 with size 241